# MEstimate Encoding categorical Features

In [1]:
import numpy as np
import pandas as pd
import string, copy, time
import optuna

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from category_encoders import MEstimateEncoder
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
train_df = pd.read_csv("../../input/train.csv")
test_df = pd.read_csv("../../input/test.csv")

In [3]:
train_df.shape, test_df.shape

((900000, 33), (700000, 32))

In [4]:
train_df.head(2)

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,-2.540739,0.766952,-2.730628,-0.208177,1.363402,ABABDADBAB,67.609153,0,0,0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,2.278315,-0.633658,-1.217077,-3.782194,-0.058316,ACACCADCEB,377.096415,0,0,1


# Splitting feature 27 intelligently

By itself, we can see that feature 27 has close to 741k unique values in a train size of 900k entries. However, we can decompose these into a set of columns and then create counts of the alphabets as a feature.

In [5]:
train_df.f_27.nunique()

741354

In [6]:
# Define a default counter for all the alphabets
default_counter = {}
for s in string.ascii_uppercase:
    default_counter[s] = 0

In [7]:
# Define a function to get the alphabet counts in each record of the train/test df
def get_counts(x):
    counts = copy.deepcopy(default_counter)
    for letter in x:
        counts[letter] += 1
    return list(counts.values())

In [8]:
train_alphabet_counts = train_df.f_27.apply(lambda x: get_counts(x))
test_alphabet_counts = test_df.f_27.apply(lambda x: get_counts(x))

In [9]:
train_alphabet_df = pd.DataFrame(np.array(train_alphabet_counts.values.tolist()), columns = list(string.ascii_uppercase))
test_alphabet_df = pd.DataFrame(np.array(test_alphabet_counts.values.tolist()), columns = list(string.ascii_uppercase))

In [10]:
test_alphabet_df = pd.DataFrame(np.array(test_alphabet_counts.values.tolist()), columns = list(string.ascii_uppercase))

In [11]:
train_alphabet_df.describe().T

,count,mean,std,min,25%,50%,75%,max
A,900000.0,2.460909,1.348131,0.0,2.0,2.0,3.0,9.0
B,900000.0,3.244169,1.422739,0.0,2.0,3.0,4.0,10.0
C,900000.0,1.478987,1.063639,0.0,1.0,1.0,2.0,7.0
D,900000.0,1.028972,0.929393,0.0,0.0,1.0,2.0,7.0
E,900000.0,0.597529,0.734487,0.0,0.0,0.0,1.0,5.0
F,900000.0,0.315301,0.547126,0.0,0.0,0.0,1.0,5.0
G,900000.0,0.163871,0.398628,0.0,0.0,0.0,0.0,4.0
H,900000.0,0.094059,0.301564,0.0,0.0,0.0,0.0,3.0
I,900000.0,0.066087,0.251902,0.0,0.0,0.0,0.0,3.0
J,900000.0,0.055603,0.230243,0.0,0.0,0.0,0.0,3.0


We can see that `U, V, W, X, Y, Z` never appear in the corpus. Only upto T, we find some representation of these alphabets in our feature, so we will only keep these counts till alphabet T in our corpus and add them to the original dataframe.

In [12]:
train_df = pd.concat([train_df, train_alphabet_df.iloc[:, :20]], axis = 1)
test_df = pd.concat([test_df, test_alphabet_df.iloc[:, :20]], axis = 1)

In [13]:
train_df.head(3)

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,K,L,M,N,O,P,Q,R,S,T
0,0,-1.373246,0.238887,-0.243376,0.567405,-0.647715,0.839326,0.113133,1,5,...,0,0,0,0,0,0,0,0,0,0
1,1,1.697021,-1.710322,-2.230332,-0.545661,1.113173,-1.552175,0.447825,1,3,...,0,0,0,0,0,0,0,0,0,0
2,2,1.681726,0.616746,-1.027689,0.810492,-0.609086,0.113965,-0.708660,1,0,...,1,0,0,0,0,0,0,0,0,0


In [14]:
test_df.head(3)

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,K,L,M,N,O,P,Q,R,S,T
0,900000,0.442517,0.174380,-0.999816,0.762741,0.186778,-1.074775,0.501888,6,6,...,0,1,0,0,0,0,0,0,0,0
1,900001,-0.605598,-0.305715,0.627667,-0.578898,-1.750931,1.355550,-0.190911,1,3,...,0,0,0,0,0,0,0,0,0,0
2,900002,0.303990,2.445110,0.246515,0.818248,0.359731,-1.331845,1.358622,3,3,...,1,0,0,0,0,0,0,0,0,0


In [15]:
train_df.drop(columns = ["f_27", "id"], inplace = True)
test_ids = test_df["id"]
test_df.drop(columns = ["f_27", "id"], inplace = True)

## Fit the encoder

In [16]:
y = train_df.pop("target")
X = train_df
X_train_enc, X_encode, y_train_enc, y_encode = train_test_split(X, y, test_size = 0.2, random_state = 257)

In [17]:
cat_cols = [f"f_{fnum:0>2d}" for fnum in range(7, 19)] + ["f_29" , "f_30"]
num_cols = [y for y in X_train_enc.columns if (not y in cat_cols) and (y != "id") and (y != "f_27")]

In [18]:
encoder = MEstimateEncoder(cols = cat_cols, m = 1.5)
encoder.fit(X_encode, y_encode);

In [19]:
X_train_enc = encoder.transform(X_train_enc)
X_encode = encoder.transform(X_encode)
X_test = encoder.transform(test_df)

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_enc, y_train_enc, stratify = y_train_enc, test_size = 100000, random_state = 121)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((620000, 50), (620000,), (100000, 50), (100000,))

In [21]:
y_train.value_counts()

0    318308
1    301692
Name: target, dtype: int64

In [22]:
X_train_enc.head(2)

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,K,L,M,N,O,P,Q,R,S,T
229487,0.703040,1.246406,0.160750,-1.129079,0.415793,0.437626,-0.520668,0.480970,0.482350,0.500603,...,0,0,0,0,0,0,1,0,0,0
614596,0.478763,-0.316379,1.470314,0.401741,-1.752594,-0.294004,-0.927445,0.491573,0.506162,0.518572,...,0,0,0,0,0,0,0,0,0,0


In [25]:
def objective(trial):
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvalid = xgb.DMatrix(X_valid, label=y_valid)
    
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "booster": "gbtree",
        "lambda": trial.suggest_float("lambda", 0.5, 2.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 9, step=2),
        "n_estimators": trial.suggest_int("n_estimators", 100, 500, step=50)
    }

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    
    met = roc_auc_score(y_valid, preds)
    return met

In [27]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=150, timeout=7200)

[I 2022-05-29 16:08:28,475] A new study created in memory with name: no-name-75c09ff8-a741-4ac7-a655-b90988de478e
[I 2022-05-29 16:08:34,613] Trial 0 finished with value: 0.8098522426747787 and parameters: {'lambda': 1.2038086541359891, 'max_depth': 5, 'n_estimators': 350}. Best is trial 0 with value: 0.8098522426747787.
[I 2022-05-29 16:08:42,878] Trial 1 finished with value: 0.839095733719787 and parameters: {'lambda': 0.6240289647702126, 'max_depth': 7, 'n_estimators': 300}. Best is trial 1 with value: 0.839095733719787.
[I 2022-05-29 16:08:47,508] Trial 2 finished with value: 0.7271233666868891 and parameters: {'lambda': 0.5455072047664217, 'max_depth': 3, 'n_estimators': 100}. Best is trial 1 with value: 0.839095733719787.
[I 2022-05-29 16:08:53,924] Trial 3 finished with value: 0.8098572534737349 and parameters: {'lambda': 1.520732076842635, 'max_depth': 5, 'n_estimators': 400}. Best is trial 1 with value: 0.839095733719787.
[I 2022-05-29 16:09:06,501] Trial 4 finished with value

In [31]:
trial = study.best_trial
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

    lambda: 0.6165858719264813
    max_depth: 9
    n_estimators: 350


In [ ]:
# Predict on test data
pd.DataFrame({"id":test_ids,
              "target":model.predict_proba(X_test)[:, 1]}).to_csv("../../submissions/May_2022/submission_XGB_MEstimator_encoding.csv", index = None)

In [ ]:
# !kaggle competitions submit -c tabular-playground-series-may-2022 -f ../../submissions/May_2022/submission_XGB_MEstimator_encoding.csv -m "XGBoost with Target Encoding Submission"

# Use the entire training set

In [35]:
final_X_train = pd.concat([X_train, X_valid, X_encode])
final_y_train = pd.concat([y_train, y_valid, y_encode])

model = XGBClassifier(**trial.params, use_label_encoder = False)

final_model = model.fit(final_X_train, final_y_train)

In [40]:
# Predict on test data
pd.DataFrame({"id": test_ids,
              "target":final_model.predict_proba(X_test)[:, 1]}).to_csv("../../submissions/May_2022/submission_XGB_target_encoding_full.csv", index = None)

In [41]:
# Predict on test data
pd.DataFrame({"id": test_ids,
              "target":final_model.predict_proba(X_test)[:, 1]})#.to_csv("../../submissions/May_2022/submission_XGB_target_encoding_full.csv", index = None)

,id,target
0,900000,0.999554
1,900001,0.969786
2,900002,0.000009
3,900003,0.044755
4,900004,0.901859
...,...,...
699995,1599995,0.560077
699996,1599996,0.970752
699997,1599997,0.494648
699998,1599998,0.006546


In [42]:
!kaggle competitions submit -c tabular-playground-series-may-2022 -f ../../submissions/May_2022/submission_XGB_target_encoding_full.csv -m "XGBoost with Target Encoding Submission"

100%|██████████████████████████████████████| 12.6M/12.6M [00:07<00:00, 1.68MB/s]
Successfully submitted to Tabular Playground Series - May 2022